# 1.Import Packages and Functions

In [ ]:
%pip install python-utils

In [ ]:
# Import necessary packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

from keras.models import load_model

# # import utils
# from public_tests import *
# from test_utils import *

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# 2.Load Data

In [ ]:
train_df = pd.read_csv("/content/train-small.csv")
valid_df = pd.read_csv("/content/valid-small.csv")

test_df = pd.read_csv("/content/test.csv")

train_df.head()

In [ ]:
# data types and null values check
train_df.info()

In [ ]:
train_df.describe()

Explore Data Labels

In [ ]:
# create list of the names of each condition or disease
columns = train_df.keys()
columns = list(columns)
columns

In [ ]:
# remove unecessary elements
columns.remove('Image')
columns.remove('PatientId')

# get total number of classes
print(f"Number of columns: {len(columns)} for these conditions: {columns}")

In [ ]:
# print number of positive labels for each class
for column in columns:
  print(f"Class {column}: {train_df[column].sum()} samples")

In [ ]:
class_counts = train_df.sum().drop(['Image', 'PatientId'])
for column in class_counts.keys():
  print(f"The class {column} has {train_df[column].sum()} samples")

In [ ]:
# Plot up the distribution of counts
sns.barplot(class_counts.values, class_counts.index, color='b')
plt.title('Distribution of Classes for Training Dataset', fontsize=15)
plt.xlabel('Number of Patients', fontsize=15)
plt.ylabel('Diseases', fontsize=15)
plt.show()

In [ ]:
# check patient id
print(f"Total patient IDs: {train_df['PatientId'].count()}, Total unique IDs: {train_df['PatientId'].value_counts().shape[0]}")

Number of unique patients is fewer that number of total patients

To avoid having class imbalance impact the loss function is to weight the losses differently


In [ ]:
labels = ['Cardiomegaly',
          'Emphysema',
          'Effusion',
          'Hernia',
          'Infiltration',
          'Mass',
          'Nodule',
          'Atelectasis',
          'Pneumothorax',
          'Pleural_Thickening',
          'Pneumonia',
          'Fibrosis',
          'Edema',
          'Consolidation']

## Prevent Data Leakage
Dataset contains multiple images for each patient\
Split is done on patient level do there is no 'leakage' between train, validation and test datasets.

In [ ]:
# check for leakage
def check_for_leakage (df1, df2, patient_col):
  # convert patient columns to sets of unique identifiers
  df1_patients_unique = set(df1[patient_col])
  df2_patients_unique = set(df2[patient_col])

  # check for intersection between data sets
  patients_in_both_groups = df1_patients_unique.intersection(df2_patients_unique)

  # leakage contrains true if there is patient overlap, otherwise false
  leakage = bool(patients_in_both_groups)

  return leakage

In [ ]:
print("leakage between train and valid: {}".format(check_for_leakage(train_df, valid_df, 'PatientId')))
print("leakage between train and test: {}".format(check_for_leakage(train_df, test_df, 'PatientId')))
print("leakage between valid and test: {}".format(check_for_leakage(valid_df, test_df, 'PatientId')))

## Prepare Images
- use `ImageDataGenerator` class
- class supports basic data augmentation
- transform batch values so that mean is 0 and s.deviation is 1 (standardizing input distribution)
- convert grayscale images to 3-channel format (pre-trained model requires 3-channel input)

In [ ]:
def get_train_generator(df, image_dir, x_col, y_cols, shuffle=True, batch_size=8, seed=1, target_w = 320, target_h = 320):
    """
    Return generator for training set, normalizing using batch
    statistics.

    Args:
      train_df (dataframe): dataframe specifying training data.
      image_dir (str): directory where image files are held.
      x_col (str): name of column in df that holds filenames.
      y_cols (list): list of strings that hold y labels for images.
      batch_size (int): images per batch to be fed into model during training.
      seed (int): random seed.
      target_w (int): final width of input images.
      target_h (int): final height of input images.

    Returns:
        train_generator (DataFrameIterator): iterator over training set
    """
    print("getting train generator...")
    # normalize images
    image_generator = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization= True)

    # flow from directory with specified batch size
    # and target image size
    generator = image_generator.flow_from_dataframe(
            dataframe=df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            target_size=(target_w,target_h))

    return generator

### Build separate generator for valid and test sets
- does not use batch statistics with test and validation data. images are processed one at a time
- normalize incoming test data using statistics from training set

In [ ]:
def get_test_and_valid_generator(valid_df, test_df, train_df, image_dir, x_col, y_cols, sample_size=100, batch_size=8, seed=1, target_w = 320, target_h = 320):
    """
    Return generator for validation set and test set using
    normalization statistics from training set.

    Args:
      valid_df (dataframe): dataframe specifying validation data.
      test_df (dataframe): dataframe specifying test data.
      train_df (dataframe): dataframe specifying training data.
      image_dir (str): directory where image files are held.
      x_col (str): name of column in df that holds filenames.
      y_cols (list): list of strings that hold y labels for images.
      sample_size (int): size of sample to use for normalization statistics.
      batch_size (int): images per batch to be fed into model during training.
      seed (int): random seed.
      target_w (int): final width of input images.
      target_h (int): final height of input images.

    Returns:
        test_generator (DataFrameIterator) and valid_generator: iterators over test set and validation set respectively
    """
    print("getting train and valid generators...")
    # get generator to sample dataset
    raw_train_generator = ImageDataGenerator().flow_from_dataframe(
        dataframe=train_df,
        directory=IMAGE_DIR,
        x_col="Image",
        y_col=labels,
        class_mode="raw",
        batch_size=sample_size,
        shuffle=True,
        target_size=(target_w, target_h))

    # get data sample
    batch = raw_train_generator.next()
    data_sample = batch[0]

    # use sample to fit mean and std for test set generator
    image_generator = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization= True)

    # fit generator to sample from training data
    image_generator.fit(data_sample)

    # get test generator
    valid_generator = image_generator.flow_from_dataframe(
            dataframe=valid_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            seed=seed,
            target_size=(target_w,target_h))

    test_generator = image_generator.flow_from_dataframe(
            dataframe=test_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            seed=seed,
            target_size=(target_w,target_h))
    return valid_generator, test_generator

In [ ]:
IMAGE_DIR = "/content/images-small/"
train_generator = get_train_generator(train_df, IMAGE_DIR, "Image", labels)

valid_generator, test_generator = get_test_and_valid_generator(valid_df, test_df, IMAGE_DIR, "Image", labels)


`__get_item__(index)` function to look at what generator gives the model during training and validation

In [ ]:
x, y = train_generator.__get_item__(0)
plt.imshow(x[0]);

# 3.Model Development

## Class Imbalance
- ideally, model is trained using evely balanced dataset so that +ve and -ve training cases contribute equally to the loss
- if normal cross-entropy loss function with higliy unbalanced dataset, algo will prioritize the majority class (ie, -ve in this case) since it contributes more to the loss
> Contribution of each class (ie +ve or -ve):
  1. freq(p) = No. +ve examples / N
  2. freq(n) = No. -ve examples / N


In [ ]:
# plot frequency of labels in dataset
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(train_generator.labels, axis=0))
plt.title("Frequency of Each Class")
plt.show()

calculate class fequencies

In [ ]:
def compute_class_freqs(labels):
    """
    Compute positive and negative frequencies for each class.

    Args:
        labels (np.array): matrix of labels, size (num_examples, num_classes)

    Returns:
        positive_frequencies (np.array): array of size (num_classes) with
                                         the frequency of positive examples for each class
        negative_frequencies (np.array): array of size (num_classes) with
                                         the frequency of negative examples for each class
    """

    # Calculate the total number of patients (rows)
    N = labels.shape[0]

    # Compute positive frequencies
    positive_frequencies = np.sum(labels, axis=0) / N

    # Compute negative frequencies
    negative_frequencies = 1 - positive_frequencies

    return positive_frequencies, negative_frequencies

compute frequencies for training data

In [ ]:
freq_pos, freq_neg = compute_class_freqs(train_generator.labels)
freq_pos

visualize contribution ratios for each of the pathologies

In [ ]:
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": freq_pos})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} for l,v in enumerate(freq_neg)], ignore_index=True)
plt.xticks(rotation=90)
f = sns.barplot(x="Class", y="Value", hue="Label" ,data=data)

- above plot shows that contribution of +ve classes is significantly lower than that of the -ve classes.
- the contributions should be equal
- one way to do this can be done by multiplying each example from each class by a class-specific weight factor (w-pos and w-neg) so that overall contribution of each class is the same
> w(pos) * freq(p) = w(neg) * freq(n)\
> which can be done by taking: \
    1. w(pos) = freq(n)
    2. w(neg) = freq(p)


In [ ]:
pos_weights = freq_neg
neg_weights = freq_pos
pos_contribution = freq_pos * pos_weights
neg_contribution = freq_neg * neg_weights

In [ ]:
# graph contributions to verify this
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": pos_contribution})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v}
                        for l,v in enumerate(neg_contribution)], ignore_index=True)
plt.xticks(rotation=90)
sns.barplot(x="Class", y="Value", hue="Label" ,data=data);

- above plot shows that, by applying these weightings, the +ve and -ve lavbels in each class would have the same aggregate contribution to the loss function.
- implement loss function after computing weights

## Get Weighted Loss
- loss function calculates the weightes loss for each batch.
- multi-class loss: add avg loss for each individual class
- add small value, e, to predicted values to avoid numerical error that would occur if the predicted values happen to be zero

In [ ]:
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    """
    Return weighted loss function given negative weights and positive weights.

    Args:
      pos_weights (np.array): array of positive weights for each class, size (num_classes)
      neg_weights (np.array): array of negative weights for each class, size (num_classes)

    Returns:
      weighted_loss (function): weighted loss function
    """
    def weighted_loss(y_true, y_pred):
        """
        Return weighted loss value.

        Args:
            y_true (Tensor): Tensor of true labels, size is (num_examples, num_classes)
            y_pred (Tensor): Tensor of predicted labels, size is (num_examples, num_classes)
        Returns:
            loss (float): overall scalar loss summed across all classes
        """
        # initialize loss to zero
        loss = 0.0

        ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

        for i in range(len(pos_weights)):
            # for each class, add average weighted loss for that class
                loss += K.mean(-(pos_weights[i] *y_true[:,i] * K.log(y_pred[:,i] + epsilon)
                             + neg_weights[i]* (1 - y_true[:,i]) * K.log( 1 - y_pred[:,i] + epsilon))) #complete this line, USE Keras functions instead of Numpy
        return loss

        ### END CODE HERE ###
    return weighted_loss

## DenseNet121
- pre-trained DenseNet121 model from Keras
- add 2 layers on top:
> 1. `GlobalAveragePooling2D` layer to get the avg of the last conv later from DenseNet121
  2. `Dense` layer with `sigmoid` activation to get prediction logits for each class.

- custom loss function in `compile` function

In [ ]:
# create the base pre-trained model
base_model = DenseNet121(weights='/content/densenet.hdf5', include_top=False)

x = base_model.output

# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)

# and a logistic layer
predictions = Dense(len(labels), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss=get_weighted_loss(pos_weights, neg_weights))

# 4.Training
- training on small subset of data
- ensure loss on training set is decreasing

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=valid_generator,
                              steps_per_epoch=100,
                              validation_steps=25,
                              epochs = 3)

plt.plot(history.history['loss'])
plt.ylabel("loss")
plt.xlabel("epoch")
plt.title("Training Loss Curve")
plt.show()

## Training on larger dataset
- option: load set of pre-trained weights
- The model architecture for our pre-trained model is exactly the same, but we used a few useful Keras "callbacks" for this training. Do spend time to read about these callbacks at your leisure as they will be very useful for managing long-running training sessions:
> 1. You can use `ModelCheckpoint` callback to monitor your model's `val_loss` metric and keep a snapshot of your model at the point.
  2. You can use the `TensorBoard` to use the Tensorflow Tensorboard utility to monitor your runs in real-time.
  3. You can use the `ReduceLROnPlateau` to slowly decay the learning rate for your model as it stops getting better on a metric such as `val_los`s to fine-tune the model in the final steps of training.
  4. You can use the `EarlyStopping` callback to stop the training job when your model stops getting better in it's validation loss. You can set a `patience` value which is the number of epochs the model does not improve after which the training is terminated. This callback can also conveniently restore the weights for the best metric at the end of training to your model.
  
- (Read more about keras callbacks)

In [ ]:
model.load_weights("/content/pretrained_model.h5")

# 5.Prediction & Evaluation
- use `predict_generator` function ot generate predictions for images in test set


In [ ]:
predicted_vals = model.predict_generator(test_generator, steps = len(test_generator))

## ROC Curve and AUROC
- ROC: Receiver Operating Characteristic curve
- AUC: Area Under Curve

- Simple terms: curve that is more to the left and the top has more 'area' under it indicates that the model is performing better.

- sklearn functions:
  - `roc_curve`
  - `roc_auc_score`




---
- compare performance to the AUCs reported in the original paper:
  - [CheXNet](https://arxiv.org/abs/1711.05225)
  - [CheXpert](https://arxiv.org/pdf/1901.07031)
  - [CheXNeXt](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1002686)



In [ ]:
auc_rocs = util.get_roc_curve(labels, predicted_vals, test_generator)

## Visualize Learning with GradCAM
- One challenge using DL in medicine:
  - complex architecture used for neural networks makes them much harder to interpret compared to traditional ML models (eg linear models)
- Class Actiation Maps (CAM):
  - Common approach to increase the model interpretability.
  - Useful to understand where the model is 'looking' when classifying an image


- GradCAM technique:
  - Produce heatmap highlighting the important regions in the image for predicting the pathological condition
  - Done by extracting gradients of each predicted class
  - Does not provide explanation of reasoning for each classification probability
  - Still useful for 'debugging' model and augmenting prediction


In [ ]:
# load small training set and look at 4 classes with highest AUC measures
df = pd.read_csv("/content/train-small.csv")
IMAGE_DIR = "/content/images-small/"

# only show the labels with top 4 AUC
labels_to_show = np.take(labels, np.argsort(auc_rocs)[::-1])[:4]

Look at specific images

In [ ]:
util.compute_gradcam(model, '00008270_015.png', IMAGE_DIR, df, labels, labels_to_show)

In [ ]:
util.compute_gradcam(model, '00011355_002.png', IMAGE_DIR, df, labels, labels_to_show)

In [ ]:
util.compute_gradcam(model, '00029855_001.png', IMAGE_DIR, df, labels, labels_to_show)

In [ ]:
util.compute_gradcam(model, '00005410_000.png', IMAGE_DIR, df, labels, labels_to_show)